# Support Functions Demo

This notebook demonstrates the usage of support functions for convex sets:
- `BallSupportFunction`: For balls B(c, r)
- `EllipsoidSupportFunction`: For ellipsoids E(c, r, A)

In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
from pygeoinf.hilbert_space import EuclideanSpace
from pygeoinf.linear_operators import DiagonalSparseMatrixLinearOperator
from pygeoinf.convex_analysis import BallSupportFunction, EllipsoidSupportFunction

## 1. Setup: 2D Euclidean Space

In [2]:
# Create a 2D Euclidean space
H = EuclideanSpace(2)

# Define a center point
center = H.from_components(np.array([1.0, 0.5]))

print(f"Space dimension: {H.dim}")
print(f"Center: {H.to_components(center)}")

Space dimension: 2
Center: [1.  0.5]


## 2. Ball Support Function

For a ball $B(c, r) = \{x : \|x - c\| \leq r\}$, the support function is:

$$h(q) = \langle q, c \rangle + r \|q\|$$

In [3]:
# Create a ball with radius 2.0
radius = 2.0
ball_support = BallSupportFunction(H, center, radius)

# Test with a direction vector q
q = H.from_components(np.array([1.0, 0.0]))  # Direction along x-axis

# Evaluate support function
h_q = ball_support(q)
print(f"Direction q: {H.to_components(q)}")
print(f"Support function h(q): {h_q:.4f}")

# Get support point (extreme point in direction q)
x_star = ball_support.support_point(q)
print(f"Support point x*: {H.to_components(x_star)}")
print(f"Expected: center + r*(q/||q||) = {H.to_components(center)} + 2.0*[1, 0] = [3.0, 0.5]")

Direction q: [1. 0.]
Support function h(q): 3.0000
Support point x*: [3.  0.5]
Expected: center + r*(q/||q||) = [1.  0.5] + 2.0*[1, 0] = [3.0, 0.5]


In [4]:
# Test with another direction
q2 = H.from_components(np.array([0.0, 1.0]))  # Direction along y-axis

h_q2 = ball_support(q2)
x_star2 = ball_support.support_point(q2)

print(f"Direction q: {H.to_components(q2)}")
print(f"Support function h(q): {h_q2:.4f}")
print(f"Support point x*: {H.to_components(x_star2)}")
print(f"Expected: [1.0, 2.5]")

Direction q: [0. 1.]
Support function h(q): 2.5000
Support point x*: [1.  2.5]
Expected: [1.0, 2.5]


## 3. Ellipsoid Support Function

For an ellipsoid $E(c, r, A) = \{x : \langle A(x-c), (x-c) \rangle \leq r^2\}$ with $A$ SPD, the support function is:

$$h(q) = \langle q, c \rangle + r \|A^{-1/2} q\|$$

We use a diagonal operator for simplicity (axis-aligned ellipse).

In [5]:
# Create a diagonal operator A = diag([4.0, 1.0])
# This creates an ellipse compressed along x-axis
diag_values = np.array([4.0, 1.0])
A = DiagonalSparseMatrixLinearOperator.from_diagonal_values(H, H, diag_values)

print(f"Operator A diagonal: {diag_values}")
print(f"This means: 4*(x-1)^2 + (y-0.5)^2 <= r^2")

Operator A diagonal: [4. 1.]
This means: 4*(x-1)^2 + (y-0.5)^2 <= r^2


In [6]:
# For DiagonalSparseMatrixLinearOperator, we can use .inverse and .sqrt properties
A_inv = A.inverse
A_inv_sqrt = A_inv.sqrt

print(f"A^-1 diagonal: {A_inv.extract_diagonal()}")
print(f"A^-1/2 diagonal: {A_inv_sqrt.extract_diagonal()}")

A^-1 diagonal: [0.25 1.  ]
A^-1/2 diagonal: [0.5 1. ]


In [7]:
# Create ellipsoid support function
ellipsoid_radius = 1.0
ellipsoid_support = EllipsoidSupportFunction(
    H,
    center,
    ellipsoid_radius,
    A,
    inverse_operator=A_inv,
    inverse_sqrt_operator=A_inv_sqrt
)

# Test with direction q = [1, 0]
q = H.from_components(np.array([1.0, 0.0]))

h_q = ellipsoid_support(q)
x_star = ellipsoid_support.support_point(q)

print(f"Direction q: {H.to_components(q)}")
print(f"Support function h(q): {h_q:.4f}")
print(f"Support point x*: {H.to_components(x_star)}")

Direction q: [1. 0.]
Support function h(q): 1.5000
Support point x*: [1.5 0.5]


In [8]:
# Test with direction q = [0, 1]
q2 = H.from_components(np.array([0.0, 1.0]))

h_q2 = ellipsoid_support(q2)
x_star2 = ellipsoid_support.support_point(q2)

print(f"Direction q: {H.to_components(q2)}")
print(f"Support function h(q): {h_q2:.4f}")
print(f"Support point x*: {H.to_components(x_star2)}")

Direction q: [0. 1.]
Support function h(q): 1.5000
Support point x*: [1.  1.5]


## 4. Comparison: Ball vs Ellipsoid

Let's compare support functions in multiple directions.

In [9]:
# Test in 8 directions around the circle
angles = np.linspace(0, 2*np.pi, 8, endpoint=False)

print("Direction (angle) | Ball h(q) | Ellipsoid h(q)")
print("-" * 50)

for angle in angles:
    q = H.from_components(np.array([np.cos(angle), np.sin(angle)]))

    h_ball = ball_support(q)
    h_ellipsoid = ellipsoid_support(q)

    print(f"{np.degrees(angle):6.1f}° | {h_ball:9.4f} | {h_ellipsoid:14.4f}")

Direction (angle) | Ball h(q) | Ellipsoid h(q)
--------------------------------------------------
   0.0° |    3.0000 |         1.5000
  45.0° |    3.0607 |         1.8512
  90.0° |    2.5000 |         1.5000
 135.0° |    1.6464 |         0.4370
 180.0° |    1.0000 |        -0.5000
 225.0° |    0.9393 |        -0.2701
 270.0° |    1.5000 |         0.5000
 315.0° |    2.3536 |         1.1441


## 5. Properties of Support Functions

Support functions have useful properties:
1. **Positive homogeneity**: $h(\alpha q) = \alpha h(q)$ for $\alpha \geq 0$
2. **Subadditivity**: $h(q_1 + q_2) \leq h(q_1) + h(q_2)$

In [10]:
# Test positive homogeneity
q = H.from_components(np.array([1.0, 1.0]))
alpha = 2.5

h_q = ball_support(q)
h_alpha_q = ball_support(H.multiply(alpha, q))

print("Positive homogeneity test:")
print(f"h(q) = {h_q:.4f}")
print(f"h({alpha}*q) = {h_alpha_q:.4f}")
print(f"{alpha}*h(q) = {alpha * h_q:.4f}")
print(f"Equal? {np.isclose(h_alpha_q, alpha * h_q)}")

Positive homogeneity test:
h(q) = 4.3284
h(2.5*q) = 10.8211
2.5*h(q) = 10.8211
Equal? True


In [11]:
# Test subadditivity
q1 = H.from_components(np.array([1.0, 0.0]))
q2 = H.from_components(np.array([0.0, 1.0]))
q_sum = H.add(q1, q2)

h_q1 = ball_support(q1)
h_q2 = ball_support(q2)
h_sum = ball_support(q_sum)

print("Subadditivity test:")
print(f"h(q1) = {h_q1:.4f}")
print(f"h(q2) = {h_q2:.4f}")
print(f"h(q1 + q2) = {h_sum:.4f}")
print(f"h(q1) + h(q2) = {h_q1 + h_q2:.4f}")
print(f"h(q1 + q2) <= h(q1) + h(q2)? {h_sum <= h_q1 + h_q2}")

Subadditivity test:
h(q1) = 3.0000
h(q2) = 2.5000
h(q1 + q2) = 4.3284
h(q1) + h(q2) = 5.5000
h(q1 + q2) <= h(q1) + h(q2)? True


## 6. Without Inverse Operators (Ellipsoid)

If you don't provide inverse operators, the ellipsoid support function will raise an error when evaluated.

In [12]:
# Create ellipsoid support without inverse operators
ellipsoid_no_inv = EllipsoidSupportFunction(
    H,
    center,
    ellipsoid_radius,
    A
    # No inverse_operator or inverse_sqrt_operator provided
)

# Try to get support point (returns None)
q = H.from_components(np.array([1.0, 0.0]))
x_star = ellipsoid_no_inv.support_point(q)
print(f"Support point without inverse: {x_star}")

# Try to evaluate support function (raises error)
try:
    h_q = ellipsoid_no_inv(q)
except ValueError as e:
    print(f"\nError when evaluating h(q): {e}")

Support point without inverse: None

Error when evaluating h(q): inverse_sqrt_operator must be provided to evaluate the support function. Pass A^{-1/2} when constructing EllipsoidSupportFunction.
